In [1]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom

In [67]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = '/Users/amattfel/Documents/GitHub'
proj_dir = '/mattfeld_2020/sourcedata'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/7-fMRI_DistortionMap_AP/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
data_files = glob(data_dir + '/*')


In [ ]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
data_set = dicom.dcmread(data_files['WHAT GOES HERE'])

In [68]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(len('WHAT GOES HERE')))
#print('The timing of the TR was: {0} ms'.format(data_set.'WHAT GOES HERE'))
#print('The timing of the TE was: {0} ms'.format(???))
#print('The flip angle in degrees was: {0}'.format(???))
#print('The name of the scan was: {0}'.format(???))

# Why use the following format to access information in the header?
#print(data_set[int('00181312', 16)].value)
#print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value



The number of TRs collected equals: 66
The timing of the TR was: 6720 ms
The timing of the TE was: 80 ms
The flip angle in degrees was: 90
The name of the scan was: fMRI_DistortionMap_AP


In [38]:
# How do I find phase encoding direction information?
import nibabel.nicom.csareader as csareader
csa_tr = csareader.read(csa_str)
csa_str = data_set[int('00291010', 16)].value
print(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0])


6
292
288
33
286
172
285
287
246
259
272
148
293
311
213
206
162
214
9
44
147
244
4
96
190
220
239
351
92
89
137
258
171
314
298
198
333
197
135
161
282
350
250
32
157
180
217
84
49
90
228
58
38
249
85
164
279
234
301
318
327
328
93
284
88
216
97
10
149
65
145
248
48
316
91
102
186
335
17
195
323
124
347
307
169
70
192
25
16
261
280
260
263
219
326
235
73
271
236
117
63
153
184
19
118
305
270
109
139
128
278
37
72
322
5
60
257
342
86
266
341
23
13
349
51
22
41
75
310
79
64
3
223
344
262
312
337
140
166
188
28
69
187
61
176
247
240
15
238
227
108
167
208
59
343
2
267
115
106
196
179
300
274
66
174
47
205
221
175
229
212
354
1
218
182
231
253
12
254
76
193
155
142
156
8
224
101
241
133
74
291
107
185
34
40
348
127
211
230
264
237
150
329
138
233
62
151
120
281
321
42
299
210
317
130
100
83
204
81
116
110
43
339
134
46
265
36
294
99
159
336
209
170
242
68
256
177
202
345
52
290
24
303
332
199
123
77
55
30
168
183
35
11
111
54
201
296
121
113
105
194
39
57
95
71
78
129
276
132
331
215
338
